In [1]:
import pygame
import sys
import random

ModuleNotFoundError: No module named 'pygame'

In [2]:
#defining basic game constants
WIDTH,HEIGHT=800,600
BALL_R=20
PFOM_WD,PFOM_HT=100,10
FPS=60
BLACK=(0,0,0)
WHITE=(255,255,255)
RED=(255,0,0)
YELLOW=(255,255,0)
GREEN=(150,255,120)
BLUE=(160,225,255)
GRAY=(80,80,80)
LG=(200,200,200)
GOLD=(220,220,55)

In [3]:
#init game
pygame.init()

#init screen/instance window
window=pygame.display.set_mode(WIDTH,HEIGHT)
text=pygame.font.Font(None,30)

#setting a clock
clock=pygame.time.Clock()

#init game variables
b_pos=[ WIDTH // 2 , HEIGHT // 2 ]
b_speed=[random.uniform(2,4),random.uniform(2,4)]

pf_pos=[ (WIDTH-PFOM_WD) // 2 , HEIGHT-PFOM_HT-10 ]
pfom_speed=10
score=0
level=1
pf_color=BLUE
hearts=3

NameError: name 'pygame' is not defined

In [ ]:
#
def start_screen():
    screen.fill(BLACK)
    print_to_screen("BOUNCE",65,HEIGHT//2)
    print_to_screen("PRESS ANY KEY TO START...", 25, HEIGHT // 5)
    print_to_screen("USE ARROWS TO MOVE THE PLATFORM...", 25, HEIGHT // 5)
    pygame.display.flip()
    getch()

def endscreen():
    screen.fill(GRAY)
    print_to_screen("GAME OVER",65,HEIGHT//2,RED)
    print_to_screen("PRESS ANY KEY TO START...", 25, HEIGHT // 5)
    print_to_screen("USE ARROWS TO MOVE THE PLATFORM...", 25, HEIGHT // 5)
    pygame.display.flip()
    getch()


def winscreen():
    screen.fill(LG)
    print_to_screen("CONGRATULATIONS!!!",65,HEIGHT//2,GOLD)
    print_to_screen("YOU WIN!!! NOW GET A LIFE!!!", 25, HEIGHT // 5,GOLD)
    print_to_screen("PRESS ANY KEY TO END OR ANY OTHER KEY TO RESTART...", 25, HEIGHT // 5)
    pygame.display.flip()
    getch()

    
def getch():
    state=True
    while state:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            else:
                state=False
    